In [1]:
import dotenv
dotenv.load_dotenv()

True

# Part 10 Logical and Semantic routing

In [3]:
from typing import Literal

from langchain_core. prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

class RouteQuery(BaseModel):
    """Route a user query to the most revevant datasourse."""
    datasource: Literal['python_docs', 'js_docs', 'golang_docs'] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question"
    )

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
structured_llm = llm.with_structured_output(RouteQuery)

system = """You are an expert at running a user question to the appropriate data source.

Based on the programing language the quesion is refering to, route it to the relevant data source."""
prompt = ChatPromptTemplate.from_messages(
    [
        ('system',system),
        ('human',"{question}"),
    ]

)
router = prompt | structured_llm

In [8]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [10]:
result.datasource

'python_docs'

In [11]:
def chose_route(result: RouteQuery):
    if "python_docs" in result.datasource.lower():
        return "chain for python docs"
    elif "js_docs" in result.datasource.lower():
        return "chain for js_docs"
    else:
        return "golang_docs"
from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(chose_route)

In [12]:
full_chain.invoke({"question":question})

'chain for python docs'

## Semantic routing

In [13]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

def prompt_router(input):
    query_embeddings = embeddings.embed_query(input['query'])
    similarity = cosine_similarity([query_embeddings], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("USING MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

chain = (
    {'query': RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)
print(chain.invoke("What is a black hole"))

Using PHYSICS
Ah, a black hole! A fascinating and mind-bending object.

In simple terms, a black hole is a region in spacetime where gravity is so strong that nothing, not even light, can escape from it.

Think of it this way:

*   **Massive Compression:** Imagine taking a huge amount of matter and squeezing it into an incredibly tiny space. This creates an immense concentration of mass.
*   **Gravity's Grip:** This immense concentration of mass warps spacetime around it, creating a gravitational pull that is inescapable beyond a certain point called the *event horizon*.
*   **The Point of No Return:** The event horizon is like a boundary. Anything that crosses it is doomed to be pulled into the singularity at the center of the black hole, where our current understanding of physics breaks down.

So, a black hole is essentially a region of spacetime with such strong gravity that nothing can escape its grasp. They are formed from the remains of very massive stars or through other extreme

# Query Structuring

In [ ]:
!pip install youtube-transcript-api

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
    --------------------------------------- 0.0/2.2 MB 1.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/2.2 MB 2.6 MB/s eta 0:00:01
   --------- ------------------------------ 0.5/2.2 MB 3.9 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.2 MB 5.9 MB/s eta 0:00:01
   ---------------------------- ----------- 1.5/2.2 MB 7.0 MB/s eta 0:00:01
   -------------------------------------- - 2.1/2.2 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------  2.2/2.2 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------  2.2/2.2 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------  2.2/2.2 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------  2.2/2.2 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.5 MB/s eta 0:00:00


In [18]:
!pip install pytube

   ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
   -------------- ------------------------- 20.5/57.6 kB 640.0 kB/s eta 0:00:01
   ----------------------------------- ---- 51.2/57.6 kB 650.2 kB/s eta 0:00:01
   ----------------------------------- ---- 51.2/57.6 kB 650.2 kB/s eta 0:00:01
   ----------------------------------- ---- 51.2/57.6 kB 650.2 kB/s eta 0:00:01
   ----------------------------------- ---- 51.2/57.6 kB 650.2 kB/s eta 0:00:01
   ---------------------------------------- 57.6/57.6 kB 178.0 kB/s eta 0:00:00


In [27]:
%pip install --upgrade --quiet  yt_dlp
%pip install --upgrade --quiet  pydub
%pip install --upgrade --quiet  librosa

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [39]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=HdafI0t3sEY"
).load()

docs[0].metadata

ParseError: no element found: line 1, column 0 (<string>)